#### 1. 분류 Classification
- KNN은 classification 과 Regression 알고리즘으로 활용가능하다. 먼저 분류부터!

## 1. 데이터준비

In [82]:
# X,y나누기
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data = pd.read_csv('breast-cancer-wisconsin.csv')
X = data[data.columns[1:10]]
y = data[['Class']]

In [83]:
# train, test data 나누기
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, stratify=y, random_state=42)

In [84]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

## 2. 모델적용 KNN

In [85]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.984375

In [86]:
# 오차행렬
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train) #예측한 모델이랑 y_train 데이터랑 일치하는지 확인
print("훈련데이터 오차행렬:\n", confusion_train)

훈련데이터 오차행렬:
 [[331   2]
 [  6 173]]


In [87]:
# 평가지표 확인
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print("분류예측 레포트:\n", cfreport_train)

분류예측 레포트:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       333
           1       0.99      0.97      0.98       179

    accuracy                           0.98       512
   macro avg       0.99      0.98      0.98       512
weighted avg       0.98      0.98      0.98       512



In [88]:
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9532163742690059

In [89]:
confusion_test = confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬:\n", confusion_test)

테스트데이터 오차행렬:
 [[106   5]
 [  3  57]]


In [90]:
cfreport_test = classification_report(y_test, pred_test)
print("분류예측 레포트:\n", cfreport_test)

분류예측 레포트:
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       111
           1       0.92      0.95      0.93        60

    accuracy                           0.95       171
   macro avg       0.95      0.95      0.95       171
weighted avg       0.95      0.95      0.95       171



## 3. Grid Search 
#### 하이퍼파라미터 직접설정

In [91]:
param_grid = {'n_neighbors': [1,3,5,7,9,11]}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]})

In [92]:
print("Best Parameter:{}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter:{'n_neighbors': 3}
Best Score: 0.9824
TestSet Score: 0.9532


## 4. Random Search
#### 무작위추출

In [93]:
from scipy.stats import randint
param_distribs = {'n_neighbors': randint(low=1, high=20)} # n_neighbors를 1~20 에서 20번 무작위 추출
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(KNeighborsClassifier(),
                                  param_distributions = param_distribs,
                                  n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=20,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd44477b6d0>})

In [94]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'n_neighbors': 4}
Best Score: 0.9804
TestSet Score: 0.9415


#### 2. 회귀 Regression

In [43]:
data2 = pd.read_csv('house_price.csv')
#data2.shape
#display(data2)
X = data2[data2.columns[1:5]]
y = data2[['house_value']]

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [46]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [47]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.6804607237174459

In [48]:
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.5541889571372401

train 데이터 정확도는 68%, test 데이터 정확도는 55.4% 인것으로 보면 train 데이터에 과대적합된 K(=5) 값 일수도 있다.

In [53]:
# RMSE 
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("train data RMSE:", np.sqrt(MSE_train))
print("test data RMSE:", np.sqrt(MSE_test))

train data RMSE: 53952.69804097723
test data RMSE: 63831.91662964773


오차가 작을수록 좋은 모델이므로 train data 에 더 적합된 결과이다.

In [72]:
# Grid Search
param_grid = {'n_neighbors': [1,3,5,7,9,11]}
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]})

In [73]:
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'n_neighbors': 11}
Best Score: 0.5638
TestSet Score: 0.5880


In [74]:
# Random Search
param_distribs = {'n_neighbors': randint(low=1, high=20)}
random_search = RandomizedSearchCV(KNeighborsRegressor(),
                                  param_distributions = param_distribs,
                                  n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(), n_iter=20,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd444a5e850>})

In [75]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'n_neighbors': 19}
Best Score: 0.5777
TestSet Score: 0.6004


- 결론 :
K값을 늘려서 가까운 주변의 여러 데이터의 평균으로 주택가격을 예측하는 것이 모델의 성능 및 일반화에 도움이 되는 것으로 보인다.